In [1]:
# Originally made by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings)
# The original BigGAN+CLIP method was by https://twitter.com/advadnoun

import argparse
import math
import random
from urllib.request import urlopen
from tqdm import tqdm
import sys
import os

sys.path.append('taming-transformers')

from omegaconf import OmegaConf
from taming.models import cond_transformer, vqgan

import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from torch.cuda import get_device_properties
torch.backends.cudnn.benchmark = False

from torch_optimizer import DiffGrad, AdamP

from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio

from PIL import ImageFile, Image, PngImagePlugin, ImageChops
ImageFile.LOAD_TRUNCATED_IMAGES = True

from subprocess import Popen, PIPE
import re

# Supress warnings
import warnings

In [2]:
#AudioCLIP

import glob

import librosa
import librosa.display

import simplejpeg

import torchvision as tv

import matplotlib.pyplot as plt

from PIL import Image
from IPython.display import Audio, display

sys.path.append('AudioCLIP')

from model import AudioCLIP
from utils.transforms import ToTensor1D

torch.set_grad_enabled(False)

MODEL_FILENAME = 'AudioCLIP-Full-Training.pt'
# derived from ESResNeXt
SAMPLE_RATE = 44100
# derived from CLIP
IMAGE_SIZE = 224
IMAGE_MEAN = 0.48145466, 0.4578275, 0.40821073
IMAGE_STD = 0.26862954, 0.26130258, 0.27577711

# VQGAN

In [3]:
# Various functions and classes
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))


def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()


def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]

class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward

    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)

replace_grad = ReplaceGrad.apply


class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)

    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None

clamp_with_grad = ClampWithGrad.apply


def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)


class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))

    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow # not used with pooling
        
        # Pick your own augments & their order
        augment_list = []
        #for item in args.augments[0]:
        #    if item == 'Ji':
        #        augment_list.append(K.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.7))
        #    elif item == 'Sh':
        #        augment_list.append(K.RandomSharpness(sharpness=0.3, p=0.5))
        #    elif item == 'Gn':
        #       augment_list.append(K.RandomGaussianNoise(mean=0.0, std=1., p=0.5))
        #    elif item == 'Pe':
        #        augment_list.append(K.RandomPerspective(distortion_scale=0.7, p=0.7))
        #    elif item == 'Ro':
        #        augment_list.append(K.RandomRotation(degrees=15, p=0.7))
        #    elif item == 'Af':
        #        augment_list.append(K.RandomAffine(degrees=15, translate=0.1, shear=5, p=0.7, padding_mode='zeros', keepdim=True)) # border, reflection, zeros
        #    elif item == 'Et':
        #       augment_list.append(K.RandomElasticTransform(p=0.7))
        #    elif item == 'Ts':
        #        augment_list.append(K.RandomThinPlateSpline(scale=0.8, same_on_batch=True, p=0.7))
        #    elif item == 'Cr':
        #        augment_list.append(K.RandomCrop(size=(self.cut_size,self.cut_size), pad_if_needed=True, padding_mode='reflect', p=0.5))
        #   elif item == 'Er':
        #       augment_list.append(K.RandomErasing(scale=(.1, .4), ratio=(.3, 1/.3), same_on_batch=True, p=0.7))
        #   elif item == 'Re':
        #       augment_list.append(K.RandomResizedCrop(size=(self.cut_size,self.cut_size), scale=(0.1,1),  ratio=(0.75,1.333), cropping_mode='resample', p=0.5))
                
        self.augs = nn.Sequential(*augment_list)
        self.noise_fac = 0.1
        # self.noise_fac = False

        # Uncomment if you like seeing the list ;)
        # print(augment_list)
        
        # Pooling
        self.av_pool = nn.AdaptiveAvgPool2d((self.cut_size, self.cut_size))
        self.max_pool = nn.AdaptiveMaxPool2d((self.cut_size, self.cut_size))

    def forward(self, input):
        cutouts = []
        
        for _ in range(self.cutn):            
            # Use Pooling
            cutout = (self.av_pool(input) + self.max_pool(input))/2
            cutouts.append(cutout)
            
        batch = self.augs(torch.cat(cutouts, dim=0))
        
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch

def load_vqgan_model(config_path, checkpoint_path):
    global gumbel
    gumbel = False
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
        gumbel = True
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model

def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

In [4]:
audios = "cat.wav"
iterations = 500
clip_model = 'ViT-B/32'
vqgan_config = f'checkpoints/vqgan_imagenet_f16_16384.yaml'
vqgan_checkpoint = f'checkpoints/vqgan_imagenet_f16_16384.ckpt'
seed = None
optimiser = 'Adam'
output = "output.png"
cuda_device = "cuda:0"
learning_rate = 0.1
cut_power = 1.0
cutn = 32
size = [256, 256]
display_freq = 50

In [5]:
device = torch.device(cuda_device)
model = load_vqgan_model(vqgan_config, vqgan_checkpoint).to(device)
jit = True if "1.7.1" in torch.__version__ else False

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Restored from checkpoints/vqgan_imagenet_f16_16384.ckpt


In [6]:
aclp = AudioCLIP(pretrained=f'AudioCLIP/assets/{MODEL_FILENAME}').eval().requires_grad_(False).to(device)

In [7]:
cut_size = aclp.visual.input_resolution
f = 2**(model.decoder.num_resolutions - 1)

In [8]:
make_cutouts = MakeCutouts(cut_size, cutn, cut_pow=cut_power)

toksX, toksY = size[0] // f, size[1] // f
sideX, sideY = toksX * f, toksY * f

In [9]:
e_dim = model.quantize.e_dim
n_toks = model.quantize.n_e
z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

In [10]:
#Init Z
one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()

z = one_hot @ model.quantize.embedding.weight
z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2) 

z_orig = z.clone()
z.requires_grad_(True)

tensor([[[[ 4.0844e-05, -3.0700e-05,  3.6681e-05,  ..., -1.4597e-05,
           -4.9732e-05,  4.4859e-05],
          [ 7.3042e-06, -3.2018e-05, -6.1201e-06,  ...,  8.8099e-06,
           -4.4160e-05,  2.4237e-06],
          [ 5.5243e-05,  5.8542e-05, -5.3747e-06,  ...,  7.8748e-06,
            1.8017e-05, -1.3857e-05],
          ...,
          [ 5.6919e-05,  5.2440e-05,  6.0577e-05,  ..., -6.6900e-06,
           -2.8678e-05, -4.9574e-05],
          [ 2.0480e-05, -3.7644e-05, -5.3792e-05,  ...,  2.4722e-05,
            4.3521e-05, -2.0962e-05],
          [ 2.5345e-05, -4.5527e-05,  1.5314e-05,  ..., -4.6717e-05,
            5.1577e-05, -5.8620e-05]],

         [[ 3.8280e-05,  5.4798e-05,  2.4109e-05,  ..., -4.1992e-05,
            3.1731e-05, -3.1132e-05],
          [ 2.6099e-05, -1.0279e-06,  8.6275e-07,  ...,  1.5192e-05,
           -5.6242e-05,  3.6846e-05],
          [ 2.4202e-05,  5.1277e-05,  5.5697e-06,  ...,  5.8037e-05,
           -3.4898e-05,  4.1430e-05],
          ...,
     

In [11]:
opt = optim.Adam([z], lr=learning_rate)

# AudioCLIP

In [12]:
audio_transforms = ToTensor1D()

image_transforms = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Resize(IMAGE_SIZE, interpolation=Image.BICUBIC),
    tv.transforms.CenterCrop(IMAGE_SIZE),
    tv.transforms.Normalize(IMAGE_MEAN, IMAGE_STD)
])

/data/scratch/ngop/.envs/audio-clip/lib/python3.7/site-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [13]:
def read_data(img_dir, audio_dir):
    img_dir += "*.jpg"
    audio_dir += "*.wav"
    
    paths_to_audio = glob.glob(audio_dir)

    audio = list()
    for path_to_audio in paths_to_audio:
        track, _ = librosa.load(path_to_audio, sr=SAMPLE_RATE, dtype=np.float32)
        # compute spectrograms using trained audio-head (fbsp-layer of ESResNeXt)
        # thus, the actual time-frequency representation will be visualized
        spec = aclp.audio.spectrogram(torch.from_numpy(track.reshape(1, 1, -1)))
        spec = np.ascontiguousarray(spec.numpy()).view(np.complex64)
        pow_spec = 10 * np.log10(np.abs(spec) ** 2 + 1e-18).squeeze()

        audio.append((track, pow_spec))
    
    paths_to_images = glob.glob(img_dir)

    images = list()
    for path_to_image in paths_to_images:
        with open(path_to_image, 'rb') as jpg:
            image = simplejpeg.decode_jpeg(jpg.read())
            images.append(image)
    return images, audio

In [14]:
def show_audio(audio):
    fig, axes = plt.subplots(2, len(audio), figsize=(20, 5), dpi=100)

    for idx in range(len(audio)):
        track, pow_spec = audio[idx]

        # draw the waveform
        librosa.display.waveplot(track, sr=SAMPLE_RATE, ax=axes[0, idx], color='k')
        # show the corresponding power spectrogram
        axes[1, idx].imshow(pow_spec, origin='lower', aspect='auto', cmap='gray', vmin=-180.0, vmax=20.0)

        # modify legend
        axes[0, idx].set_title(os.path.basename(paths_to_audio[idx]))
        axes[0, idx].set_xlabel('')
        axes[0, idx].set_xticklabels([])
        axes[0, idx].grid(True)
        axes[0, idx].set_ylim(bottom=-1, top=1)

        axes[1, idx].set_xlabel('Time (s)')
        axes[1, idx].set_xticks(np.linspace(0, pow_spec.shape[1], len(axes[0, idx].get_xticks())))
        axes[1, idx].set_xticklabels([f'{tick:.1f}' if tick == int(tick) else '' for tick in axes[0, idx].get_xticks()])
        axes[1, idx].set_yticks(np.linspace(0, pow_spec.shape[0] - 1, 5))

    axes[0, 0].set_ylabel('Amplitude')
    axes[1, 0].set_ylabel('Filter ID')

    plt.show()
    plt.close(fig)

    for idx, path in enumerate(paths_to_audio):
        print(os.path.basename(path))
        display(Audio(audio[idx][0], rate=SAMPLE_RATE, embed=True))

In [15]:
def embedding(text = None, images = None, audio = None):
    text_features = []
    image_features = []
    audio_features = []
    
    if audio != None: 
        audio = torch.stack([audio_transforms(track.reshape(1, -1)) for track, _ in audio])
        ((audio_features, _, _), _), _ = aclp(audio=audio)
        audio_features = audio_features / torch.linalg.norm(audio_features, dim=-1, keepdim=True)
    if images != None:
        images = torch.stack([image_transforms(image) for image in images])
        ((_, image_features, _), _), _ = aclp(image=images)
        image_features = image_features / torch.linalg.norm(image_features, dim=-1, keepdim=True)
    if text != None:
        text = [[label] for label in text]
        ((_, _, text_features), _), _ = aclp(text=text)
        text_features = text_features / torch.linalg.norm(text_features, dim=-1, keepdim=True)
    return text_features, image_features, audio_features

In [16]:
aclp = AudioCLIP(pretrained=f'AudioCLIP/assets/{MODEL_FILENAME}').eval().requires_grad_(False)
_, audio = read_data("AudioCLIP/demo/images/", "AudioCLIP/demo/sound/cutted/")
audio = [audio[0]]

In [20]:
print(len(audio[0]))

2


In [17]:
_, _, audio_rep = embedding(audio = audio)

/data/scratch/ngop/.envs/audio-clip/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [18]:
out = synth(z)
iii = aclp.encode_image(normalize(make_cutouts(out))).float()

print(iii.shape)
print(F.normalize((iii).unsqueeze(1), dim=2).shape)

print(pMs[0].embed.shape)
print(pMs[0].embed.unsqueeze(0).shape)

NameError: name 'synth' is not defined

In [19]:
text_rep, _, _ = embedding(text = ["hey"])

AudioCLIP/model/audioclip.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_indices = torch.tensor(batch_indices).int()


In [20]:
print(text_rep.shape)

torch.Size([1, 1024])


In [21]:
pMs = []
normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                  std=[0.26862954, 0.26130258, 0.27577711])

pMs.append(Prompt(audio_rep).to(device))

# Run

In [22]:
aclp = AudioCLIP(pretrained=f'AudioCLIP/assets/{MODEL_FILENAME}').eval().requires_grad_(False).to(device)

In [23]:
def synth(z):
    if gumbel:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embed.weight).movedim(3, 1)
    else:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

In [24]:
@torch.inference_mode()
def checkin(i, losses):
    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
    out = synth(z)
    info = PngImagePlugin.PngInfo()
    TF.to_pil_image(out[0].cpu()).save(output, pnginfo=info) 

In [25]:
def ascend_txt():
    global i
    out = synth(z)
    iii = aclp.encode_image(normalize(make_cutouts(out))).float()
    
    result = []

    for prompt in pMs:
        result.append(prompt(iii))

    return result # return loss

In [26]:
def train(i):
    opt.zero_grad(set_to_none=True)
    lossAll = ascend_txt()
    
    if i % display_freq == 0:
        checkin(i, lossAll)
       
    loss = sum(lossAll)
    loss.backward()
    opt.step()
    
    #with torch.no_grad():
    with torch.inference_mode():
        z.copy_(z.maximum(z_min).minimum(z_max))

In [27]:
# Do it

i = 0 # Iteration counter

try:
    with tqdm() as pbar:
        while True:            
            train(i)
            if i == max_iterations:
                break
            i += 1
            pbar.update()
except KeyboardInterrupt:
    pass

0it [00:00, ?it/s]

i: 0, loss: 1.03718, losses: 1.03718


0it [00:00, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn